# NB #1:  Pool Exploration 

### Pool address: 0x8b6e6E7B5b3801FEd2CaFD4b22b8A16c2F2Db21a  -- 80% WETH - 20% DAI 

The Balancer Pool cadCAD model provides a core infrastructure for simulating Balancer Pools in the **Token Engineering Design and Verification process**.

It allows to simulate Balancer Pool transactions and verify new designs and application cases.  
In this notebook we plug **historical on-chain data** to a cadCAD model, and analyse **Pool Power** and **Pool Characteristics**.

For more information check out the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/).

# A. System Context

### A1.1 System Specification
System specification details are available in the Balancer Simulations documentation:
- [Differential Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Mathematical Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition/balancer_math.py)
- [Model Architecture](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Naming Convention](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/naming-convention)

# B. cadCAD Simulations

### B1.1 Dependencies

In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

### B1.2 Initialize Pool


In this section you specify the Balancer Pool to be analyzed and run scripts to produce the'genesis_state', based on on-chain data. 

More information here: [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/onchaintransactions).  

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from decimal import Decimal

# Spot price reference must be a symbol of a token in the pool in ALL_CAPS, you can ignore the spot price parameter for the simulations in this notebook.
parameters = {
    'spot_price_reference': ['DAI'],
}


#### Import Pool Transactions and Initialize Pool state

Genesis state, pool transactions, and (external) USD token prices are pulled from on-chain/API data collected in a .json file.  
**Choose *your* Balancer Pool, and produce this .json file first, then add the file path below.**

Learn more how to create this file using parsing scripts in the [documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/additional-code-and-instructions/onchaintransactions).


In [ ]:
from model.genesis_states import generate_initial_state

initial_values = generate_initial_state(initial_values_json='data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-initial_pool_states-prices.json', spot_price_base_currency=parameters['spot_price_reference'][0])

#### State Variables and Initial Values

In [ ]:
print('## State Variables')
print('# Pool')
pool = initial_values['pool']
pp.pprint(initial_values)


#### External USD Token Prices

In [ ]:
print('# External token prices, initial state')
token_prices = initial_values['token_prices']
pp.pprint(token_prices)

print('# Action Type')
action_type = initial_values['action_type']
pp.pprint(action_type)


### B1.3 State Update Functions & Policies

Balancer Simulations replicate Balancer Pool Transactions in state update functions.
For a detailed description on how we implement transactions, please visit the [Balancer Simulations documentation/Balancer Pool Functions](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/additional-code-and-instructions/balancer-the-python-edition/balancer-pool-functions).

### B1.4 Partial State Update Blocks

Partial State Update Blocks combine the following steps:  
1. **Initialize pool**, generate genesis state
2. Compute subsequent **actions in discrete timesteps**, store datetime and update the pool state variables
3. Update external **USD prices in discrete timesteps**, and store datetime  

For more information please visit the [Balancer Simulations documentation/V1.0 Model Overview](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/balancer-simulations/v10nboverview).  

To inject historical on-chain transactions to the model, reference the **actions.json of your pool below**.

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks('data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-actions-prices.json')
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### B1.5 Configuration

[cadCAD simulations](https://github.com/cadCAD-org/cadCAD/blob/master/documentation/README.md) support Monte Carlo runs, and parameter sweeps which are not applied in this notebook and can be ignored here.  
Timesteps 'T' are defined by your pool's .json file, no need to make edits here.

In [ ]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps
        'M': parameters,  # simulation parameters
    }
)

### B1.6 Execution

In [ ]:
from model.sim_runner import *

df = run(initial_values, partial_state_update_blocks, sim_config)

### B1.7 Simulation Output Preparation

Post-processing (utils.py) splits up state variable dictionaries, and adds metrics to the data frame, such as 
- `token_total_value` (TVL)
- `generated_fees_(tokensymbol)` (fee collected per transaction)
- `token_k_values`  (token value in USD)  
based on  
- `token_k_balances`
- `token_k_price`  



In [ ]:
from model.parts.utils import post_processing

p_df = post_processing(df, include_spot_prices=False)

p_df.head()

# C. Simulation Outcome & Pool Exploration

Below we show a range of plots exploring pool states in the simulation:

**a) Pool Power:**  
- C1.1 TVL (Total Value Locked, over time) compared to  
- C1.1 Pool Growth (token balances in the pool, over time)  
- C1.2 Token Balances (individual token balances, over time)

**b) Pool Characteristics:**  
- C1.3 Source of Pool Growth
- C1.4 Token Ratio
- C1.5 Action Types
- C1.6 Arbitrage Gaps (Bid/Ask Spread AMM vs. external markets)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
print("Observation Time")
print(f"Start: {(p_df.iloc[0]['change_datetime'])}")
print(f"End: {(p_df.iloc[-1]['change_datetime'])}")
print("\n")
print(f"Total No. of Timesteps (incl. Price Updates): {(p_df.iloc[-1]['timestep'])}")
print(f"Total No. of Transactions: {len(p_df[p_df.action_type != 'external_price_update'])}")

### C1.1 TVL & Pool Size Growth

In [ ]:
#TVL vs. Pool Size Growth
tvl = p_df[['timestep', 'tvl', 'total_token_balances']] 
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=tvl.timestep, y=tvl.tvl, line=dict(color='#f36315'), name="TVL total_token_value"), secondary_y=False)
fig.add_trace(go.Scatter(x=tvl.timestep,y=tvl.total_token_balances, line=dict(color='#2C1839'), name="total_token_balance"), secondary_y=True)
fig.update_layout(title_text="<b>TVL vs. Pool Growth</b>")
fig.update_xaxes(title_text="timestep")
fig.update_yaxes(title_text="<b>TVL total_token_value</b> in USD", secondary_y=False)
fig.update_yaxes(title_text="<b>total_token_balance</b> in #", secondary_y=True)
fig.show()  

### C1.2 Token Balances

In [ ]:
#balance growth % (total observation period)
dai_growth = ((p_df.iloc[-1][['token_dai_balance']])-(p_df.iloc[0][['token_dai_balance']]))*100/(p_df.iloc[0][['token_dai_balance']])
weth_growth = ((p_df.iloc[-1][['token_weth_balance']])-(p_df.iloc[0][['token_weth_balance']]))*100/(p_df.iloc[0][['token_weth_balance']])
growth = list(zip(dai_growth, weth_growth)) 
g_df = pd.DataFrame(growth, columns = ['dai_growth','weth_growth' ]).transpose(copy=True).reset_index()
g_df.columns =['token', 'growth']
print(g_df)
#plot
fig = make_subplots(rows=2, cols=2, subplot_titles=("DAI Growth", "Balance Growth in %", "WETH Growth"))
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_dai_balance, line=dict(color='#4675ed'), name='token_dai_balance'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_weth_balance, line=dict(color='#4145ab'), name='token_weth_balance'), row=2, col=1)
fig.add_trace(go.Bar(y=g_df.growth, x=g_df.token, marker_color=['#4675ed','#4145ab']), row=1, col=2)
fig.update_layout(height=600, width=1000, showlegend=False, title_text="<b>Balance Growth</b> in observation period")
fig.show()

### C1.3 Sources of Growth 

In [ ]:
#Pool Shares
ps = p_df[['timestep','pool_shares']]
fig = px.line(ps, x=ps.timestep,y=ps.pool_shares)
fig.update_layout(height=300, width=1000, title_text="<b>Pool Shares (BPT)</b>")
fig.update_xaxes(title_text="timestep")
fig.update_yaxes(title_text="<b>pool shares</b> in #", range=[99.985,101.00])
fig.show()

In [ ]:
#Fees
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_dai, line=dict(color='#4675ed'), name='fees DAI'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_weth, line=dict(color='#4145ab'), name='fees WETH'), row=2, col=1)
fig.update_layout(height=400, width=1000, title_text="<b>Fees Generated</b> per transaction")

In [ ]:
# DAI fee events vs. ETH fee events
WETH_fee_events = len(p_df[p_df.generated_fees_weth > 0])
DAI_fee_events = len(p_df[p_df.generated_fees_dai > 0])
values = [DAI_fee_events, WETH_fee_events]
labels = ['DAI fees', 'WETH fees']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', marker=dict(colors=px.colors.sequential.Turbo))
fig.update_layout(height=500, width=1000, title_text="<b>Fee generating events</b> in total observation time")
fig.show()

In [ ]:
#proportions of fee value contribution
daif = p_df.groupby('action_type').sum()['generated_fees_dai'].drop(['exit', 'join', 'external_price_update','pool_creation'])
wethf = p_df.groupby('action_type').sum()['generated_fees_weth'].drop(['exit', 'join', 'external_price_update','pool_creation'])
fee_v = pd.concat([daif, wethf], axis=1)

#calculate USD value (define USD current value)
fee_v['DAI_fee_value'] = fee_v.generated_fees_dai*1.002042 #define DAI price
fee_v['WETH_fee_value'] = fee_v.generated_fees_weth*633.725 #define WETH price

#add total 
fee_v = fee_v.append(fee_v.sum().rename('total'))
print(fee_v)

#plot pie chart comparing ETH fees vs DAI fees
values = [fee_v.loc['total']['DAI_fee_value'], fee_v.loc['total']['WETH_fee_value']] 
labels = ['DAI_fees', 'WETH_fees']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', marker=dict(colors=px.colors.sequential.Turbo))
fig.update_layout(height=500, width=1000, title_text="<b>Fee value contribution</b> in USD, in total observation time")
fig.show()

### C1.4 Token Ratio

In [ ]:
events = [0, -1] #Start/End of observation period
ratio = pd.DataFrame(p_df.iloc[events][['token_dai_balance','token_weth_balance']]).transpose(copy=True)
ratio.columns =['start', 'end'] 
#plot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=ratio.index, values=ratio.start), 1, 1)
fig.add_trace(go.Pie(labels=ratio.index, values=ratio.end), 1, 2)
fig.update_traces(hoverinfo="label+percent+name", marker=dict(colors=px.colors.sequential.Turbo))
fig.update_layout(height=400, width=1000, title_text="<b>Token Ratio</b> Start/End Observation Period in %")
fig.show()

### C1.5 Action Types

In [ ]:
# Show Action Types
a_p_df = p_df.groupby('action_type').size().to_frame('count').reset_index()
fig = px.pie(a_p_df, values='count', names='action_type', title='<b>Action Types</b>', hole=.3, color_discrete_sequence=px.colors.sequential.Turbo)
fig.show()
print(a_p_df)

In [ ]:
#action type per timestep
fig = px.scatter(p_df, x='timestep', y='total_token_balances', color='action_type', color_discrete_sequence=px.colors.sequential.Turbo, category_orders={'action_type': ['pool_creation', 'join', 'join_swap', 'swap', 'exit_swap', 'exit', 'external_price_update']})
fig.update_layout(height=400, width=1000, title_text="<b>Action Type / Timestep</b>")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

### C1.6 Arbitrage Gaps (Bid/Ask Spread AMM vs. external markets)

In [ ]:
#Spot Price Deviation, bid/ask spread and arbitrage opportunity
arb = p_df[['timestep', 'action_type','token_dai_price','token_weth_price']].dropna()
spotpr = pd.DataFrame(p_df['spot_prices'].values.tolist(), index=df.index).astype('float64').dropna() #spotprice from simulation output dict
arb['WETH_spot_price_in_DAI'] = spotpr
arb['WETH_spot_price_in_USD'] = arb.WETH_spot_price_in_DAI*arb.token_dai_price #WETH in DAI spotprice simulation output to USD

fig = go.Figure()
fig.add_trace(go.Scatter(x=arb.index, y=arb.WETH_spot_price_in_USD, line=dict(color='#30123b'), mode='lines+markers', name='WETH spot price')) # fill to trace0 y
fig.add_trace(go.Scatter(x=arb.index, y=arb.token_weth_price, line=dict(color='#fe9b2d'), fill='tonexty', name='WETH external price feed')) 
fig.update_layout(height=400, width=1000, title_text="<b>WETH Spot Price vs. external price feed</b> in USD")
fig.update_yaxes(title_text="<b>WETH</b> in USD")
fig.show()